# Importing some packages

In [1]:
import numpy as np
import pandas as pd

# Loading data

In [2]:
train_df = pd.read_excel('Train.xlsx')
test_df = pd.read_excel('Test.xlsx')
sub_df = pd.read_csv('Submission.csv')

# EDA

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59709 entries, 0 to 59708
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0             59709 non-null  int64         
 1   St_Hour                59709 non-null  int64         
 2   St_Day                 59709 non-null  int64         
 3   St_Month               59709 non-null  int64         
 4   St_Year                59709 non-null  int64         
 5   User_type              59709 non-null  object        
 6   Datetime               59709 non-null  datetime64[ns]
 7   Rental_Bicycles_Count  59709 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 3.6+ MB


In [5]:
train_df.head()

,Unnamed: 0,St_Hour,St_Day,St_Month,St_Year,User_type,Datetime,Rental_Bicycles_Count
0,0,0,1,1,2013,occasional,2013-01-01 00:00:00,20
1,1,0,1,1,2013,subscribed,2013-01-01 00:00:00,26
2,2,1,1,1,2013,occasional,2013-01-01 01:00:00,23
3,3,1,1,1,2013,subscribed,2013-01-01 01:00:00,23
4,4,2,1,1,2013,occasional,2013-01-01 02:00:00,20


In [28]:
train_df.tail()

,Unnamed: 0,St_Hour,St_Day,St_Month,St_Year,User_type,Datetime,Rental_Bicycles_Count
59704,59704,21,31,5,2016,subscribed,2016-05-31 21:00:00,529
59705,59705,22,31,5,2016,occasional,2016-05-31 22:00:00,298
59706,59706,22,31,5,2016,subscribed,2016-05-31 22:00:00,455
59707,59707,23,31,5,2016,occasional,2016-05-31 23:00:00,292
59708,59708,23,31,5,2016,subscribed,2016-05-31 23:00:00,450


The training data doesn't contain any missing values judging by the tables above.

In [10]:
train_df.duplicated().sum()

0

The output of the above cell confirms that there are no duplicates in the training data.

Let's check whether the difference between all subsequent dates is an hour. If not, then we have missing dates.

In [53]:
for user_type in train_df['User_type'].unique():
    filtered_df = train_df[train_df['User_type']==user_type].sort_values('Datetime', ascending=True).reset_index(drop=True)
    filtered_df['date_diff'] = filtered_df['Datetime'].diff().apply(lambda x: x if pd.isna(x) else x.seconds).to_frame()
    inconsitency_mask = (filtered_df['date_diff'].iloc[1:] != 3600)
    n_inconsistency = inconsitency_mask.sum()
    print(10*"#" + f" {user_type} " + 10*"#")
    print(f"{n_inconsistency} not subsequent dates")
    print('\n')
    print('Dates differences except for hourly')
    print(filtered_df[filtered_df['date_diff'] != 3600]['date_diff'].value_counts())
    print('\n')


########## occasional ##########
89 not subsequent dates


Dates differences except for hourly
date_diff
7200     82
10800     6
14400     1
Name: count, dtype: int64


########## subscribed ##########
43 not subsequent dates


Dates differences except for hourly
date_diff
7200     37
10800     5
14400     1
Name: count, dtype: int64




It turned out that the train data contain data for some dates. Let's find these dates and save in a dictionary.

In [35]:
train_full_dates = pd.date_range(start='2013-01-01', end='2016-06-01', freq='60min')[:-1]
train_full_dates

DatetimeIndex(['2013-01-01 00:00:00', '2013-01-01 01:00:00',
               '2013-01-01 02:00:00', '2013-01-01 03:00:00',
               '2013-01-01 04:00:00', '2013-01-01 05:00:00',
               '2013-01-01 06:00:00', '2013-01-01 07:00:00',
               '2013-01-01 08:00:00', '2013-01-01 09:00:00',
               ...
               '2016-05-31 14:00:00', '2016-05-31 15:00:00',
               '2016-05-31 16:00:00', '2016-05-31 17:00:00',
               '2016-05-31 18:00:00', '2016-05-31 19:00:00',
               '2016-05-31 20:00:00', '2016-05-31 21:00:00',
               '2016-05-31 22:00:00', '2016-05-31 23:00:00'],
              dtype='datetime64[ns]', length=29928, freq='60min')

In [48]:
missing_dates_dict = {}
for user_type in train_df['User_type'].unique():
    train_date = train_df[train_df['User_type']==user_type]['Datetime'].tolist()
    missing_dates = [i for i in train_full_dates if i not in train_date]
    missing_dates_dict[user_type] = missing_dates
    print(f"{user_type}: {len(missing_dates)} missing dates")

occasional: 97 missing dates
subscribed: 50 missing dates


In [4]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  1440 non-null   int64         
 1   St_Hour     1440 non-null   int64         
 2   St_Day      1440 non-null   int64         
 3   St_Month    1440 non-null   int64         
 4   St_Year     1440 non-null   int64         
 5   User_type   1440 non-null   object        
 6   Datetime    1440 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 78.9+ KB


In [6]:
test_df.head()

,Unnamed: 0,St_Hour,St_Day,St_Month,St_Year,User_type,Datetime
0,0,0,1,6,2016,occasional,2016-06-01 00:00:00
1,1,0,1,6,2016,subscribed,2016-06-01 00:00:00
2,2,1,1,6,2016,occasional,2016-06-01 01:00:00
3,3,1,1,6,2016,subscribed,2016-06-01 01:00:00
4,4,2,1,6,2016,occasional,2016-06-01 02:00:00


Checking whether the difference between all subsequent dates is an hour in the test set.

In [54]:
for user_type in test_df['User_type'].unique():
    filtered_df = test_df[test_df['User_type']==user_type].sort_values('Datetime', ascending=True).reset_index(drop=True)
    filtered_df['date_diff'] = filtered_df['Datetime'].diff().apply(lambda x: x if pd.isna(x) else x.seconds).to_frame()
    inconsitency_mask = (filtered_df['date_diff'].iloc[1:] != 3600)
    n_inconsistency = inconsitency_mask.sum()
    print(10*"#" + f" {user_type} " + 10*"#")
    print(f"{n_inconsistency} not subsequent dates")
    print('\n')
    print('Dates differences except for hourly')
    print(filtered_df[filtered_df['date_diff'] != 3600]['date_diff'].value_counts())
    print('\n')


########## occasional ##########
0 not subsequent dates


Dates differences except for hourly
Series([], Name: count, dtype: int64)


########## subscribed ##########
0 not subsequent dates


Dates differences except for hourly
Series([], Name: count, dtype: int64)




In [8]:
sub_df.head()

,Rental_Bicycles_Count
0,0
1,0
2,0
3,0
4,0


## Univariate analysis

### Categorical data

In [11]:
train_df['User_type'].value_counts()

User_type
subscribed    29878
occasional    29831
Name: count, dtype: int64

In [12]:
test_df['User_type'].value_counts()

User_type
occasional    720
subscribed    720
Name: count, dtype: int64